In [ ]:
# Load needed libraries

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
OECD = pd.read_csv('data/OECD_Productivity.csv')
OECD.head()

In [ ]:
df = OECD.copy()

countries = OECD['Country'].unique()
locs = OECD['LOCATION'].unique()

df = df.loc[df['Subject'].isin(('GDP per hour worked', 'Average hours worked per person employed'))]

df = df.loc[df['Measure'].isin(('USD, constant prices, 2010 PPPs', 'Persons/Hours'))]

df = df[['LOCATION', 'Subject', 'Time', 'Value']]

# Set MultiIndex; can choose ordering
df.set_index(['Subject', 'LOCATION', 'Time'], inplace=True)

# Create nested columns
df = df.unstack(0)

# Remove one unnecessary level of naming
df = df['Value']

df.columns = ['h', 'y_h']

df['cntr'] = df.index.get_level_values(0)

# Show prepared dataset
df.head()

In [ ]:
x = df['h']
y = df['y_h']

In [ ]:
plt.scatter(x, y, alpha=0.2, label='All countries')
plt.scatter(x['KOR'], y['KOR'], color='C2', alpha=0.3, label='Korea')
plt.scatter(x['POL'], y['POL'], color='C3', alpha=0.3, label='Poland')

plt.legend(frameon=False)

plt.title('GDP per hour vs average hours worked')
plt.xlabel('Average hours worked per employee')
plt.ylabel('GDP per hour (constant 2010 PPP USD)')

plt.show()

In [ ]:
for loc in locs[:9]:
    plt.scatter(np.log(x[loc]), np.log(y[loc]), alpha=0.2, label=loc)

plt.legend(frameon=False, ncol=2)

plt.title('GDP per hour vs average hours worked')
plt.ylabel('Average hours worked per employee')
plt.xlabel('GDP per hour (constant 2010 PPP USD)')

plt.show()

Utility maximization problem
\begin{align*}
\max_{c,\,h}\quad & U=\frac{c^{1-\sigma}}{1-\sigma}-\phi\frac{h^{1+\kappa}}{1+\kappa}\\
\text{subject to}\quad & y=Ah^{1-\alpha}\\
 & c=y
\end{align*}

Lagrangian
\begin{align*}
\mathcal{L}&=\frac{c^{1-\sigma}}{1-\sigma}-\phi\frac{h^{1+\kappa}}{1+\kappa}+\lambda\left[Ah^{1-\alpha}-c\right]
\end{align*}

FOCs
\begin{eqnarray*}
c & : & c^{-\sigma}-\lambda=0\\
h & : & -\phi h^{\kappa}+\lambda\left(1-\alpha\right)Ah^{-\alpha}=0
\end{eqnarray*}

Optimality condition
\begin{align*}
\left(1-\alpha\right)Ah^{-\alpha}c^{-\sigma}=\phi h^{\kappa}
\end{align*}
\begin{align*}
c&=\left[\frac{\phi}{\left(1-\alpha\right)A}h^{\alpha+\kappa}\right]^{-\frac{1}{\sigma}}&=\left[\frac{\left(1-\alpha\right)A}{\phi}\right]^{\frac{1}{\sigma}}h^{-\frac{\alpha+\kappa}{\sigma}}
\end{align*}
Plug into budget constraint and solve for optimal $h$
\begin{align*}
\left[\frac{\left(1-\alpha\right)A}{\phi}\right]^{\frac{1}{\sigma}}h^{-\frac{\alpha+\kappa}{\sigma}}&=Ah^{1-\alpha}
\end{align*}
\begin{align*}
h^{-\frac{\alpha+\kappa+\sigma\left(1-\alpha\right)}{\sigma}}&=A^{\frac{\sigma-1}{\sigma}}\left(\frac{\phi}{1-\alpha}\right)^{\frac{1}{\sigma}}
\end{align*}
\begin{align*}
h&=A^{\frac{1-\sigma}{\alpha+\kappa+\sigma\left(1-\alpha\right)}}\left(\frac{1-\alpha}{\phi}\right)^{\frac{1}{\alpha+\kappa+\sigma\left(1-\alpha\right)}}
\end{align*}
\begin{align*}
\ln h&=\frac{1-\sigma}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\ln A+\frac{1}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\ln\left(\frac{1-\alpha}{\phi}\right)
\end{align*}

Rewrite above in terms of labor productivity $y/h$
\begin{align*}
\frac{y}{h}&=Ah^{-\alpha}
\end{align*}
\begin{align*}
\ln\left(y/h\right)&=\ln A-\alpha\ln h
\end{align*}
\begin{align*}
\ln A&=\ln\left(y/h\right)+\alpha\ln h
\end{align*}
\begin{align*}
\ln h&=\frac{1-\sigma}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\left[\ln\left(y/h\right)+\alpha\ln h\right]+\frac{1}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\ln\left(\frac{1-\alpha}{\phi}\right)
\end{align*}
\begin{align*}
\left(1-\frac{\alpha-\alpha\sigma}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\right)\ln h&=\frac{1-\sigma}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\ln\left(y/h\right)+\frac{1}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\ln\left(\frac{1-\alpha}{\phi}\right)
\end{align*}
\begin{align*}
\left(\frac{\kappa+\sigma}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\right)\ln h&=\frac{1-\sigma}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\ln\left(y/h\right)+\frac{1}{\alpha+\kappa+\sigma\left(1-\alpha\right)}\ln\left(\frac{1-\alpha}{\phi}\right)
\end{align*}

End result
\begin{align*}
\ln h&=\frac{1-\sigma}{\kappa+\sigma}\ln\left(y/h\right)+\frac{1}{\kappa+\sigma}\ln\left(\frac{1-\alpha}{\phi}\right)
\end{align*}
\begin{align*}
\frac{\partial\ln h}{\partial\ln\left(y/h\right)}&=\frac{1-\sigma}{\kappa+\sigma}
\end{align*}

In [ ]:
import statsmodels.formula.api as smf

# Exclude Korea as outlier
df_no_Korea = df[df['cntr'] != 'KOR']

results = smf.ols('np.log(h) ~ np.log(y_h) + C(cntr, Treatment("OECD"))', data=df_no_Korea).fit()

results.summary()

In [ ]:
elasticity = results.params[-1]
intercept = results.params[0]

print(elasticity)

country_effects = results.params[1:-1]
country_effects.index = [e[29:-1] for e in country_effects.index]

# country_effects.index

In [ ]:
country_effects.sort_values().plot(kind='bar', color='C0', figsize=(10,5))

plt.show()

In [ ]:
country_effects.sort_values()